<img src="./images/DLI_Header.png" style="width: 400px;">

# 2. Unoptimized deployment of LLama2

In this lab, we are going to look at several different strategies for deploying large models. As for this notebook, we will start with a basic example of how to run inference for LLama2 13B. We will not implement any optimizations to the model, instead we will deploy this 13B parameter model using PyTorch and an out-of-the box Transformers library. This approach, despite currently not being the most performant option for a production system, will allow us to run our first inference requests. We will demonstrate how to use Few-Shot Learning to transform our generic language model into a neural machine translation tool to carry out English to French translation. We will conclude this notebook by measuring inference latency so that we can compare our performance to a more optimized version of the model. 

The goals of this notebook are to: 
* Deploy a 13B parameter large LLama2 model using nothing but PyTorch and the Transformers library. 
* Learn about the basics of prompt engineering which will allow us to take advantage of few-shot learning capability of large models. 
* We will also measure the speed of inference to use it as a baseline for the next sections of this lab.

**[2.1 LLama2 13B deployment with the Transformers library](#2.1)<br>**
**[2.2 Few-shot learning](#2.1)<br>**
**[2.3 Speed measurement](#2.2)<br>**

## 2.1 LLama2 13B deployment with the Transformers library
### Transformers library

The Transformers library, developed by HuggingFace, is a utility for development of transformer-based architectures for NLP, CV and other machine learning applications. It is also a community-based repository hosting thousands of pretrained models from contributors across the globe. This includes models for different modalities such as text, vision, and audio. Besides training, Transformers library can also be used for inference, including inference of large transformer-based architectures. This includes models trained with Transformer library but also external models including those trained with Megatron-LM and other libraries. In this part of the class, we will use it to deploy LLama2 and execute it on a GPU. </br> 

### LLama2 13B 

[Llama2](https://arxiv.org/abs/2307.09288) is a collection of pretrained and fine-tuned generative text models ranging in scale from 7 billion to 70 billion parameters. We will explore the 13B pretrained model. Llama 2 was pretrained on 2 trillion tokens of data from publicly available sources. 
Llama2 is an auto-regressive language model that uses an optimized transformer architecture. It uses the standard Transformer architecture, applies RMSNorm for pre-normalization, uses SwiGLU activation function, and employs rotated position embeddings (RoPE). The main differences in architecture compared to [Llama1](https://arxiv.org/abs/2302.13971) include extending the context length from 2048 to 4096 and utilizing Grouped Query Attention (GQA).

The LLaMA tokenizer is a BPE model based on sentencepiece. One quirk of sentencepiece is that when decoding a sequence, if the first token is the start of the word (e.g. “Banana”), the tokenizer does not prepend the prefix space to the string.

### The lab environment
All steps will be carried out within the Triton Docker container <b>nvcr.io/nvidia/tritonserver:23.10-trtllm-python-py3</b> (baseline for the next labs)

### Single GPU deployment

The 13B parameter model is small enough to fit into the memory of one 80GB A100. We will start with a single GPU deployment for now and move on to model parallel deployment in the next notebook. Let us begin by importing the key dependencies like PyTorch and Transformers library.

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

Let us initialize a pre-trained LLama2 13B model and the required tokenizer. The model is big, so to limit the amount of time required for its download, we have saved it in a local folder called `weights`.  We will use the `from_pretrained()` function with a local path to load the copy of the model. The commented lines show how to download the model from the HuggingFace repository. 

<b>When this Jupyter server launched, we began downloading the weights of the model in the background. If you have reached this point of the class quickly, it is possible that the download is still in progress. If you face an error in the next step that says ` We could not connect to 'https://huggingface.co' to load this model` or that model/weights can't be found, please wait a couple more minutes for the weights to finish downloading.</b> 

The 2 lines below demonstrate how to download the pretrained model on your own system. In this lab, we have pre-downloaded the weights for you.</br>
```
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-13b-hf")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf")
```

In [ ]:
# We already downloaded model weights so will use local path to the weights:
model = AutoModelForCausalLM.from_pretrained("/dli/task/weights/")
tokenizer = AutoTokenizer.from_pretrained("/dli/task/weights/")

Notes from Hugging Face [documentation](https://huggingface.co/docs/transformers/main/model_doc/llama2#usage-tips): 
The Llama2 models were trained using bfloat16, but the original inference uses float16. The checkpoints uploaded on the Hub use torch_dtype = 'float16', which will be used by the AutoModel API to cast the checkpoints from torch.float32 to torch.float16. </br>
fp16 representation allows us to take additional advantage of TensorCore acceleration of GPUs.The dtype of the online weights is mostly irrelevant unless you are using torch_dtype="auto" when initializing a model using model = AutoModelForCausalLM.from_pretrained("path", torch_dtype = "auto"). The reason is that the model will first be downloaded (using the dtype of the checkpoints online), then it will be casted to the default dtype of torch (becomes torch.float32), and finally, if there is a torch_dtype provided in the config, it will be used.</br>

We will also switch the model to an evaluation (`.eval()`) mode. Evaluation mode will allow us to remove elements of the execution graph that are not necessary for inference. It will also change the behavior of some other layers (like dropout or batch normalization) which behave differently outside of the training loop. In addition, the common practice for evaluating/validation is using `torch.no_grad()` in pair with `model.eval()` to turn off gradient computation: 

In [5]:
assert torch.cuda.is_available()
device = torch.device("cuda:0")
model.half().to(device)
model = model.eval()

Now that we have loaded our model, we are ready for inference. Since this is a generative model and we are not providing it with any prompts to guide its behavior, the model will generate <b>random</b> sentences. We will look at how to change that in just a minute.

In [6]:
# Generate a random sentence.
with torch.no_grad():
    output = model.generate(input_ids=None, max_length=128, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

The generated sentences cannot be read in their current format. We need to decode them back from tokens to text to be able to print them, using the same Tokenizer model that Llama used at Training time. 

In [ ]:
# Decoding the generated text
for sentence in output:
    sentence = sentence.tolist()
    text = tokenizer.decode(sentence, clean_up_tokenization_spaces=True)
    print(text)

## 2.2 Prompt engineering / Few shot learning.

During the lecture, we discussed the fact that the bigger the model gets, the more sample efficient it becomes. Bigger models, once pretrained, become Few-Shot learners demonstrating exceptional generalizability. With just a few samples, they can adapt to new, previously unseen tasks. Few-Shot Learning refers to the practice of feeding a machine learning model with a small amount of training data to guide its predictions. Large generative models can be provided with just a few examples of a new task at inference time not changing any model weights. This contrasts with standard fine-tuning techniques which require a large amount of training data for the pre-trained model to adapt to the desired task. </br>

Those few training examples are very frequently referred to as "Prompts". A prompt typically consists of a text describing the problem with no, one or a couple of examples of the task we want the model to carry out (hence zero, singe and few shot learning). Few-Shot Learning can be used with Large Language Models because they have learned to perform a wide number of tasks implicitly during their pre-training on large text datasets. This enables the model to generalize, that is, to understand related but previously unseen tasks with just few examples. 

Let us try to do Few-Shot inference with the LLama2 model. We will attempt to adapt our model to carry out question answering. We will achieve that by providing the model with some examples of question and expected answers, and in the final part of the prompt, we will only provide the question to be answered  aligned with the examples provided. E.g.: 

<b>"Question: What is the capital city of France? Answer:  ",</b> 

Using this prompt with multiple examples (Few-Shots), we “demonstrate” to the model what we expect to see in the generated output, and we expect that model will finalize the sentence with an answer to the provided questions. 

Let's try without any example of the expected task : 

In [ ]:
input_ids = tokenizer.encode("What is the capital city of France? ", return_tensors="pt").cuda(0)

# generate 20 new tokens
greedy_output = model.generate(input_ids=input_ids, max_new_tokens=20)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))


The model does not seem to have understood what is expected of it. Let's try with a bit of guidance on the expected format. 

In [ ]:
input_ids = tokenizer.encode("Question: What is the capital city of France? Answer: ",return_tensors="pt").cuda(0)

# generate 20 new tokens
greedy_output = model.generate(input_ids=input_ids, max_new_tokens=20)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Interesting! It understands the task as a multiple-choice question, but this is not exactly what we would like to see. We want the answer! Let's try with some given examples. 

In [ ]:
input_ids = tokenizer.encode("Question: What is the typical color of the sky ? Answer: Blue. " \
                             "Question: At what temperature does water boil? Answer: Hundred degrees celcius.  " 
                             "Question: What is the name of the first man on the moon ? Answer: Neil Amrstrong. " \
                             "Question: What is the capital city of France? Answer: ",return_tensors="pt").cuda(0)

# generate 20 new tokens
greedy_output = model.generate(input_ids=input_ids, max_new_tokens=20)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Here we go! Even better, the model continues to offer questions and answers. The task seems perfectly mastered, thanks to the help of our few examples!

## 2.3 Decoding / Quality of Output

In [ ]:
input_ids = tokenizer.encode("The sky is",return_tensors="pt").cuda(0)

# generate 40 new tokens
greedy_output = model.generate(input_ids=input_ids, max_new_tokens=40)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

This answer is grammatically correct but does not make much sense. 
This happened because we used a greedy decoder for output generation. Our model generates one token at a time, and on each generation step, we took the token with the maximum probability, leading to a suboptimal solution. Greedy decoding is one of the simplest approaches, but many different techniques exist that allow us to maximize the quality of the generation. This includes techniques such as: `Beam Search`, `Top-K` and `Top-P`. Additionally, some of those methods have hyperparameters that can be adjusted, such as `Temperature` of the logits or `Repetition penalty` to further control the quality of the generated output. 

With that in mind, let us adapt the decoding algorithm and change some of its hyperparameters. 

In [ ]:
output = model.generate(input_ids=input_ids, max_length=80, \
                        num_return_sequences=1, \
                        num_beams=5, \
                        temperature=0.7, \
                        repetition_penalty=3.0, \
                        pad_token_id=tokenizer.eos_token_id)

sentence = output[0].tolist()
text = tokenizer.decode(sentence, clean_up_tokenization_spaces=True)
print(text)

A more robust decoding algorithm creates an output of higher creativity and quality! 

Learn more about decoding methods here: [How to generate text: using different decoding methods for language generation with Transformers](https://huggingface.co/blog/how-to-generate) 

### [ Optional Task ]

In the section above, we have demonstrated just a single example of prompt engineering. If you search using your favorite search engine for "prompt engineering Llama" or "prompt examples Llama" there should be countless examples that can be adapted to your own problem. Below is an example of how to convert this model to do SQL generation. Do you think you can prompt it into writing python code or solving mathematical equations? Experiment with the code below and use as many internet resources as you want to help you get started. For more detailed information on prompting please refer to this review paper: https://arxiv.org/pdf/2107.13586.pdf. 

In [58]:
input_ids = tokenizer.encode("Create an SQL request to find all users that live in Califorian and have more than 1000 credits.", return_tensors="pt").cuda(0)

In [ ]:
output = model.generate(input_ids=input_ids, max_length=100,\
                        num_return_sequences=1,\
                        num_beams=5,\
                        temperature=0.7,\
                        repetition_penalty=3.0,\
                        pad_token_id=tokenizer.eos_token_id)
sentence = output[0].tolist()
text = tokenizer.decode(sentence, clean_up_tokenization_spaces=True)
print(text)

Example of a [solution](solutions/llama_sql.ipynb) prompt 

## 2.3 Inference latency measurement

Now let's have a look at how fast our inference pipeline is. We will measure performance of a random 128 token generation.

In [15]:
# Generate the sentence.
import time

execution_time = 0
num_iterations = 10
with torch.no_grad():
    for _ in range(num_iterations):
        start = time.time()
        output = model.generate(input_ids=None, max_length=128,\
                                 num_return_sequences=1,\
                                 pad_token_id=tokenizer.eos_token_id,\
                                 eos_token_id=50256)
        end = time.time()
        execution_time += end - start

In [16]:
print("Average inference time of 128 tokens is:",
     1000 * (execution_time/float(num_iterations)), "ms")

Average inference time of 128 tokens is: 4059.711360931397 ms


128 tokens can be generated in around 4 seconds. Let us move to the next notebook and test an optimized inference pipeline.

<h2 style="color:green;">Congratulations!</h2>

Great job finishing this notebook! Please proceed to: [Inference of the LLama2 13B model with TensorRT-LLM.](03_TRTLLMRunInferenceOfTheLLama.ipynb)
